In [ ]:
import pandas as pd
import altair as alt
import numpy as np
from meteostat import Daily, Hourly, Point
from datetime import datetime

df = pd.read_csv('MBTA.csv')
df

In [ ]:
key_bus = df[df["route_category"].astype(str).str.strip().eq("Key Bus")].copy()
key_bus.head()

In [ ]:
key_bus["service_date"] = pd.to_datetime(
    key_bus["service_date"], utc=True, errors="coerce"
)
key_bus["month"] = key_bus["service_date"].dt.to_period("M").dt.to_timestamp()

key_monthly = (
    key_bus
    .groupby(["month", "gtfs_route_id"], as_index=False)
    .agg(num=("otp_numerator", "sum"),
         den=("otp_denominator", "sum"))
)
key_monthly["on_time_rate"] = key_monthly["num"] / key_monthly["den"]
key_monthly


In [ ]:
legend_color = [
    '#1b9e77', '#d95f02', '#7570b3', '#e7298a',
    '#66a61e', '#e6ab02', '#a6761d', '#666666',
    '#1f78b4', '#b2df8a', '#fb9a99', '#cab2d6',
    '#fdbf6f', '#ff7f00', '#6a3d9a', '#b15928'
]

input_dropdown = alt.binding_select(
    options = [None, '1', '15', '22', '23', '28', '32', '39', '57', '66', 
               '71', '73', '77', '111', '116', '117'],
    labels  = ['All', 'Route 1', 'Route 15', 'Route 22', 'Route 23', 
               'Route 28', 'Route 32', 'Route 39', 'Route 57', 'Route 66', 
               'Route 71', 'Route 73', 'Route 77', 'Route 111', 'Route 116', 'Route 117'],
    name = "Bus route: "
)

route_select = alt.selection_point(fields = ['gtfs_route_id'], bind = input_dropdown)

routes_chart = alt.Chart(key_monthly).mark_line(interpolate='monotone').encode(
    alt.X('month:T', title='Month'),
    alt.Y('on_time_rate:Q',
          title='On-time rate',
          axis=alt.Axis(format='%'),
          scale=alt.Scale(domain=[0.5, 1.0], clamp=True)),
    color = alt.condition(
        route_select,
        alt.Color('gtfs_route_id:N',
                  scale=alt.Scale(range=legend_color),
                  legend=None),
        alt.value('grey')
    ),
    opacity = alt.condition(route_select, alt.value(1), alt.value(0.1)),
    tooltip = [
        alt.Tooltip('month:T', title='Month'),
        alt.Tooltip('gtfs_route_id:N', title='Route'),
        alt.Tooltip('on_time_rate:Q', title='On-time rate', format='.1%')
    ]
).add_params(
    route_select
).properties(
    title = "Key Bus Routes — Monthly On-time Rate",
    width = 900,
    height = 400
)


covid_period = pd.DataFrame({
    'start': ['2020-02-01'],
    'end':   ['2021-06-30'],
    'label': ['COVID-19 period']
})


highlight = alt.Chart(covid_period).mark_rect(
    opacity=0.2  
).encode(
    x='start:T',
    x2='end:T'
)


label = alt.Chart(covid_period).mark_text(
    align='left',
    baseline='top',
    dx=5,
    dy=5
).encode(
    x='start:T',
    y=alt.value(0),     
    text='label:N'
)


final_chart = highlight + label + routes_chart

final_chart



In [ ]:
# merge datasets

boston = Point(42.365, -71.009) 

start, end = datetime(2016,1,1), datetime(2025,12,31)


wx_daily = Daily(boston, start, end).fetch().reset_index()
wx_daily.rename(columns={'time':'date',
                         'tavg':'tavg_c','tmin':'tmin_c','tmax':'tmax_c',
                         'prcp':'prcp_mm','snow':'snow_cm','wspd':'wind_kmh'}, inplace=True)

key_bus['service_date'] = pd.to_datetime(key_bus['service_date'], utc=True, errors='coerce')\
                             .dt.tz_localize(None).dt.normalize()

wx_daily['date'] = pd.to_datetime(wx_daily['date']).dt.tz_localize(None).dt.normalize()

key_bus_weather = key_bus.merge(wx_daily, left_on='service_date', right_on='date', how='left', validate='m:1')\
              .drop(columns=['date'])

key_bus_weather.head()

In [ ]:
alt.data_transformers.disable_max_rows()

key_bus_weather["on_time_rate"] = (
    key_bus_weather["otp_numerator"] / key_bus_weather["otp_denominator"]
)

temp = alt.Chart(key_bus_weather).mark_point().encode(
    alt.Y('on_time_rate:Q', title='On-time rate', axis=alt.Axis(format='%')),
    alt.X('tavg_c:Q',  title='Temperature (°C)'), 
    tooltip=[
        'service_date:T',
        alt.Tooltip('on_time_rate:Q', title='On-time', format='.1%'),
        'tavg_c:Q'
    ]
).properties(title='Daily On-time Rate vs Temperature')

rain = alt.Chart(key_bus_weather).transform_filter('datum.prcp_mm > 0').mark_point().encode(
    alt.Y('on_time_rate:Q', title='On-time rate', axis=alt.Axis(format='%')),
    alt.X('prcp_mm:Q', title='Daily Rainfall (mm)'), 
    tooltip=[
        'service_date:T',
        alt.Tooltip('on_time_rate:Q', title='On-time', format='.1%'),
        'prcp_mm:Q'
    ]
).properties(title='Daily On-time Rate vs Rainfall')

snow = alt.Chart(key_bus_weather).transform_filter('datum.snow_cm > 0').mark_point().encode(
    alt.Y('on_time_rate:Q', title='On-time rate', axis=alt.Axis(format='%')),
    alt.X('snow_cm:Q', title='Daily Snowfall (cm)'), 
    tooltip=[
        'service_date:T',
        alt.Tooltip('on_time_rate:Q', title='On-time', format='.1%'),
        'snow_cm:Q'
    ]
).properties(title='Daily On-time Rate vs Snowfall')

wind = alt.Chart(key_bus_weather).mark_point().encode(
    alt.Y('on_time_rate:Q', title='On-time rate', axis=alt.Axis(format='%')),
    alt.X('wind_kmh:Q', title='Wind speed (km/h)'), 
    tooltip=[
        'service_date:T',
        alt.Tooltip('on_time_rate:Q', title='On-time', format='.1%'),
        'wind_kmh:Q'
    ]
).properties(title='Daily On-time Rate vs Wind Speed')

temp | rain | snow | wind
final_chart1 = temp | rain | snow | wind

final_chart1

In [ ]:
# example
rainy = key_bus_weather[key_bus_weather["prcp_mm"] > 0].copy()


rainy_route = (
    rainy
    .groupby("gtfs_route_id", as_index=False)
    .agg(
        rainy_num=("otp_numerator", "sum"),
        rainy_den=("otp_denominator", "sum")
    )
)
rainy_route["rainy_on_time_rate"] = rainy_route["rainy_num"] / rainy_route["rainy_den"]
rainy_route["gtfs_route_id"] = rainy_route["gtfs_route_id"].astype(str)

rainy_bar = alt.Chart(rainy_route).mark_bar().encode(
    x=alt.X(
        "gtfs_route_id:N",
        title="Bus route",
        sort=alt.SortField(field="rainy_on_time_rate", order="descending")
    ),
    y=alt.Y(
        "rainy_on_time_rate:Q",
        title="On-time rate on rainy days",
        axis=alt.Axis(format="%")
    ),
    tooltip=[
        alt.Tooltip("gtfs_route_id:N", title="Route"),
        alt.Tooltip("rainy_on_time_rate:Q", title="On-time rate", format=".1%"),
        alt.Tooltip("rainy_den:Q", title="Total trips on rainy days")
    ]
).properties(
    title="Key Bus Routes — On-time Rate on Rainy Days",
    width=500,
    height=300
)

rainy_bar


In [ ]:
for col in ["prcp_mm", "snow_cm"]:
    key_bus_weather[col] = pd.to_numeric(key_bus_weather[col], errors="coerce")

key_bus_weather["weather_cat"] = np.where(
    key_bus_weather["snow_cm"].fillna(0) > 0,
    "Snowy",
    np.where(
        key_bus_weather["prcp_mm"].fillna(0) > 0,
        "Rainy",
        "Clear"
    )
)

key_bus_weather["gtfs_route_id"] = key_bus_weather["gtfs_route_id"].astype(str)


weather_dropdown = alt.binding_select(
    options=["All", "Clear", "Rainy", "Snowy"],
    labels=["All weather", "Clear days", "Rainy days", "Snowy days"],
    name="Weather: "
)
weather_param = alt.param("Weather", bind=weather_dropdown, value="All")

amount_param = alt.param(
    "max_amount",
    value=50,   
    bind=alt.binding_range(
        min=1,
        max=80,   
        step=5,
        name="Max rain/snow (mm or cm): "
    )
)


base = (
    alt.Chart(key_bus_weather)
    .transform_filter(
        "Weather == 'All' ? true : datum.weather_cat == Weather"
    )
   

    .transform_filter(
        "Weather == 'Rainy' ? datum.prcp_mm <= max_amount : "
        "(Weather == 'Snowy' ? datum.snow_cm <= max_amount : true)"
    )
    
    .transform_aggregate(
        total_num="sum(otp_numerator)",
        total_den="sum(otp_denominator)",
        groupby=["gtfs_route_id"]
    )
    .transform_calculate(
        on_time_rate="datum.total_num / datum.total_den"
    )
)

bar_chart = (
    base
    .mark_bar()
    .encode(
        x=alt.X(
            "gtfs_route_id:N",
            title="Bus route",
            sort='-y'  
        ),
        y=alt.Y(
            "on_time_rate:Q",
            title="On-time rate",
            axis=alt.Axis(format="%")
        ),
        tooltip=[
            alt.Tooltip("gtfs_route_id:N", title="Route"),
            alt.Tooltip("on_time_rate:Q", title="On-time rate", format=".1%"),
            alt.Tooltip("total_den:Q", title="Total trips (after filter)")
        ]
    )
    .add_params(weather_param, amount_param)
    .properties(
        title="Average On-time Rate by Weather",
        width=600,
        height=350
    )
)

bar_chart




In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import branca.colormap as cm

routes = gpd.read_file("mbtabus", layer="MBTABUSROUTES_ARC")
routes["MBTA_ROUTE"] = routes["MBTA_ROUTE"].astype(str).str.lstrip('0')

key_ids = ['1','15','22','23','28','32','39',
           '57','66','71','73','77','111','116','117']


key_routes_gdf = routes[routes["MBTA_ROUTE"].isin(key_ids)].copy()


In [ ]:
key_routes_gdf.plot()

In [ ]:
key_bus_weather["gtfs_route_id"] = key_bus_weather["gtfs_route_id"].astype(str)


route_weather = (
    key_bus_weather
    .groupby(["gtfs_route_id", "weather_cat"], as_index=False)
    .agg(
        otp_num=("otp_numerator", "sum"),
        otp_den=("otp_denominator", "sum")
    )
)
route_weather["on_time_rate"] = route_weather["otp_num"] / route_weather["otp_den"]

key_routes_weather = key_routes_gdf.merge(
    route_weather,
    left_on="MBTA_ROUTE",     
    right_on="gtfs_route_id",  
    how="left"
)
key_routes_weather

In [ ]:
snow_map = key_routes_weather[key_routes_weather["weather_cat"] == "Snowy"].copy()

ax = snow_map.plot(
    column="on_time_rate",
    legend=True,
    cmap="RdYlGn",
    linewidth=3
)
ax.set_title("Key Bus Routes — On-time Rate on Snowy Days")
ax.set_axis_off()


In [ ]:
routes_4326 = key_routes_weather.to_crs(epsg=4326)

vmin = routes_4326["on_time_rate"].min()
vmax = routes_4326["on_time_rate"].max()

colormap = cm.LinearColormap(
    colors=[
        "#b30000",  
        "#f03b20",
        "#fd8d3c",
        "#fecc5c",
        "#ffffb2",
        "#d9f0a3",
        "#78c679",
        "#238443"   
    ],
    vmin=vmin,
    vmax=vmax
)
colormap.caption = "On-time rate"

m = folium.Map(
    location=[42.36, -71.06], 
    zoom_start=11,
    tiles="CartoDB positron"   
)


for weather in ["Clear", "Rainy", "Snowy"]:
    sub = routes_4326[routes_4326["weather_cat"] == weather]

    fg = folium.FeatureGroup(name=f"{weather} days")

    folium.GeoJson(
        sub,
        style_function=lambda feature, cmap=colormap: {
            "color": cmap(feature["properties"]["on_time_rate"]),
            "weight": 4,
            "opacity": 0.9
        },
        tooltip=folium.GeoJsonTooltip(
            fields=["MBTA_ROUTE", "on_time_rate"],
            aliases=["Route", "On-time rate"],
            localize=True,
            labels=True,
            sticky=False
        ),
        highlight_function=lambda feature: {
            "weight": 6,
            "opacity": 1.0
        }
    ).add_to(fg)

    fg.add_to(m)


colormap.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

m


In [ ]:
routes_4326 = key_routes_weather.to_crs(epsg=4326)

vmin = routes_4326["on_time_rate"].min()
vmax = routes_4326["on_time_rate"].max()

colormap = cm.LinearColormap(
    colors=[
        "#fd8d3c",
        "#fecc5c",
        "#ffffb2",
        "#d9f0a3",
        "#78c679",
        "#238443"   
    ],
    vmin=vmin,
    vmax=vmax
)
colormap.caption = "On-time rate"


m = folium.Map(
    location=[42.36, -71.06],
    zoom_start=11,
    tiles=None
)

folium.TileLayer("CartoDB positron", control=False).add_to(m)



for weather in ["Clear", "Rainy", "Snowy"]:
    sub = routes_4326[routes_4326["weather_cat"] == weather]

    
    fg = folium.FeatureGroup(
        name=f"{weather} days", 
        overlay=False, 
        show=(weather == "Clear") 
    )

    folium.GeoJson(
        sub,
        style_function=lambda feature, cmap=colormap: {
            "color": cmap(feature["properties"]["on_time_rate"]),
            "weight": 4,
            "opacity": 0.9
        },
        tooltip=folium.GeoJsonTooltip(
            fields=["MBTA_ROUTE", "on_time_rate"],
            aliases=["Route", "On-time rate"],
            localize=True,
            labels=True,
            sticky=False
        ),
        highlight_function=lambda feature: {
            "weight": 6,
            "opacity": 1.0
        }
    ).add_to(fg)

    fg.add_to(m)


colormap.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m